In [1]:
import dspy
class SummarizerSignature(dspy.Signature):
    __doc__ = """
    Summarize a document based on the difference between the old and new chunk.\n
    The Summary should be in English.\n
    The summary should be thorough and concise.\n
    Do not provide any instructions to the bot on how to perform the review.\n
    Do not mention that files need a through review or caution about potential issues\n
    The summary should not exceed 500 words.\n
    """
    old_chunk_code:str = dspy.InputField(
        desc="The code of the document",
    )
    new_chunk_code:str = dspy.InputField(
        desc="The code of the document",
    )
    summary:str = dspy.OutputField(
        desc="The summary of the document",
    )

/Users/abcom/.pyenv/versions/3.11.0/envs/reviewturtl/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from dotenv import load_dotenv
import os
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
turbo = dspy.OpenAI(
    model="gpt-4o",
    api_key=api_key,
    max_tokens=3000,
)
dspy.settings.configure(lm=turbo)

In [6]:
class DspyProgramme(dspy.Module):
    def __init__(self, signature):
        super().__init__()
        self.predictor = dspy.TypedChainOfThought(signature)

    def forward(self,context):
        return dspy.Prediction(
            answer=self.predictor(context=context)
        )

In [7]:
summarizer_programme = DspyProgramme(signature=SummarizerSignature)

In [12]:
old_chunk_code = """
def calculate_sum(numbers):
    total = 0
    for number in numbers:
        total += number
    return total
"""

new_chunk_code = """
def calculate_sum(numbers):
    if not all(isinstance(i, (int, float)) for i in numbers):
        raise ValueError("All elements must be numbers")
    total = 0
    for number in numbers:
        total += number
    return total
"""

prediction = summarizer_programme.predictor(
    old_chunk_code=old_chunk_code,
    new_chunk_code=new_chunk_code
)
summary = prediction.summary
print(summary)

The new code for the `calculate_sum` function includes an additional validation step to check that all elements in the input list are numbers (either integers or floats). If any element is not a number, the function raises a `ValueError`. This enhancement ensures that the function handles invalid inputs more gracefully, preventing potential errors during the summation process. The core functionality of summing the numbers remains the same.
